In [1]:
import numpy as np 
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
data_gen = ImageDataGenerator(zoom_range=0.2,
                              rotation_range=0.2,
                              horizontal_flip=True,
                              rescale=1/255)

In [4]:
train_data_1 = data_gen.flow_from_directory('crop_images',
                             target_size=(224,224))
train_data_2 = data_gen.flow_from_directory('kag2',
                                            target_size=(224,224))

Found 201 images belonging to 5 classes.
Found 804 images belonging to 5 classes.


In [5]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [6]:
inc_model= InceptionV3(include_top=False,input_shape=(224,224,3))

2022-11-29 17:52:44.259498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 17:52:44.260437: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 17:52:44.396694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 17:52:44.397551: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 17:52:44.398313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

87924736/87910968 [==============================] - 1s 0us/step


In [7]:
for layer in inc_model.layers:
    layer.trainable=False

In [8]:
last_layer= Dense(5,activation='softmax',)(Flatten()(inc_model.output))
model_inc= Model(inputs=inc_model.input,outputs=last_layer)
model_inc.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model_inc.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [10]:
model_inc.fit(train_data_1, epochs=15)

2022-11-29 17:53:14.684283: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2022-11-29 17:53:20.190209: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


7/7 [==============================] - 16s 502ms/step - loss: 31.9997 - accuracy: 0.1791
Epoch 2/15
7/7 [==============================] - 3s 447ms/step - loss: 11.8842 - accuracy: 0.3333
Epoch 3/15
7/7 [==============================] - 3s 415ms/step - loss: 3.9347 - accuracy: 0.6219
Epoch 4/15
7/7 [==============================] - 3s 419ms/step - loss: 5.9100 - accuracy: 0.5124
Epoch 5/15
7/7 [==============================] - 3s 379ms/step - loss: 2.2644 - accuracy: 0.7214
Epoch 6/15
7/7 [==============================] - 3s 350ms/step - loss: 3.6016 - accuracy: 0.6915
Epoch 7/15
7/7 [==============================] - 3s 403ms/step - loss: 9.0353 - accuracy: 0.5274
Epoch 8/15
7/7 [==============================] - 3s 365ms/step - loss: 1.0975 - accuracy: 0.8209
Epoch 9/15
7/7 [==============================] - 3s 361ms/step - loss: 3.5394 - accuracy: 0.6766
Epoch 10/15
7/7 [==============================] - 3s 363ms/step - loss: 1.7099 - accuracy: 0.8259
Epoch 11/15
7/7 [==========

In [11]:
model_inc.fit(train_data_2, epochs=15)

Epoch 1/15
26/26 [==============================] - 14s 554ms/step - loss: 2.2150 - accuracy: 0.8184
Epoch 2/15
26/26 [==============================] - 11s 422ms/step - loss: 0.9849 - accuracy: 0.9042
Epoch 3/15
26/26 [==============================] - 10s 398ms/step - loss: 1.2240 - accuracy: 0.8731
Epoch 4/15
26/26 [==============================] - 11s 404ms/step - loss: 0.9699 - accuracy: 0.8943
Epoch 5/15
26/26 [==============================] - 11s 429ms/step - loss: 0.8763 - accuracy: 0.9229
Epoch 6/15
26/26 [==============================] - 11s 408ms/step - loss: 0.1119 - accuracy: 0.9801
Epoch 7/15
26/26 [==============================] - 10s 398ms/step - loss: 1.0863 - accuracy: 0.9092
Epoch 8/15
26/26 [==============================] - 11s 431ms/step - loss: 0.6653 - accuracy: 0.9291
Epoch 9/15
26/26 [==============================] - 11s 408ms/step - loss: 0.1366 - accuracy: 0.9714
Epoch 10/15
26/26 [==============================] - 11s 406ms/step - loss: 1.0359 - accura

In [12]:
import cv2
def get_pred_inc(path):
    A = cv2.imread('test_crop_image/'+file)
    A = cv2.resize(A,(224,224))
    A = A.reshape(1,224,224,3)/255
    y = model_inc.predict(A).argmax()
    if y==0:
        return 'jute'
    elif y==1:
        return 'maize'
    elif y==2:
        return 'rice'
    elif y==3:
        return 'sugarcane'
    else:
        return 'wheat'

In [14]:
Test_result=os.listdir('test_crop_image')

In [15]:
import cv2

In [16]:
Output_inc=[]
for file in Test_result:
    Output_inc.append([file,get_pred_inc(file)])
Output_inc

[['maize000.jfif', 'maize'],
 ['rice-5174887_1280.jpg', 'rice'],
 ['juteplants.jpg', 'maize'],
 ['maize images.jfif', 'maize'],
 ['sugarcaneplant.jpg', 'sugarcane'],
 ['maize corn set2.jpg', 'maize'],
 ['wheat-field-artificial-irrigation-rural-electrification-to-harvest-166395991.jpg',
  'sugarcane'],
 ['juteimg.jpg', 'rice'],
 ['wheatcropfield04.jpg', 'sugarcane'],
 ['rice-4967164_1280.jpg', 'rice'],
 ['rice-fields-204128_1280.jpg', 'rice'],
 ['jute002.jpg', 'sugarcane'],
 ['wheatarial02.jpg', 'maize'],
 ['juteplant.jpg', 'maize'],
 ['sugarcane fields.jpg', 'maize'],
 ['sugarcaneplants.jpg', 'maize'],
 ['sugarcanefield02.jpg', 'wheat'],
 ['maize plant set.jpg', 'maize'],
 ['maizecornleaves.jfif', 'maize'],
 ['wheat-field01.jpg', 'rice'],
 ['wheatcropfield.jpg', 'wheat'],
 ['jute-field.jpg', 'jute'],
 ['rice-field01.jpg', 'sugarcane'],
 ['wheatcrop01.jpg', 'rice'],
 ['jutefield.jpg', 'maize'],
 ['sugarcane-field8.jpg', 'maize'],
 ['jute003.jpg', 'sugarcane'],
 ['sugarcane-farm-in-the-m